# West Nile Project

"Given weather, location, testing, and spraying data, this competition asks you to predict **when and where different species of mosquitos will test positive for West Nile virus**. A more accurate method of predicting outbreaks of West Nile virus in mosquitos will help the City of Chicago and CPHD more efficiently and effectively allocate resources towards preventing transmission of this potentially deadly virus." kaggle

## Import Libraries


In [64]:
#imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats

#additional libraries
import datetime as dt

## Import the Data

In [65]:
#long & lat for places in chi
spray = pd.read_csv('./data/spray.csv')

In [66]:
#weather data
weather = pd.read_csv('./data/weather.csv')

In [67]:
#train & test sets
test = pd.read_csv('./data/test.csv')
train = pd.read_csv('./data/train.csv')

## Cleaning Test & Train

#### Spray

In [68]:
spray.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14835 entries, 0 to 14834
Data columns (total 4 columns):
Date         14835 non-null object
Time         14251 non-null object
Latitude     14835 non-null float64
Longitude    14835 non-null float64
dtypes: float64(2), object(2)
memory usage: 463.7+ KB


In [69]:
spray.head()

,Date,Time,Latitude,Longitude
0,2011-08-29,6:56:58 PM,42.391623,-88.089163
1,2011-08-29,6:57:08 PM,42.391348,-88.089163
2,2011-08-29,6:57:18 PM,42.391022,-88.089157
3,2011-08-29,6:57:28 PM,42.390637,-88.089158
4,2011-08-29,6:57:38 PM,42.390410,-88.088858


In [70]:
spray.Date = pd.to_datetime(spray.Date)

In [71]:
#since only time has NaNs, taking a shortcut to fill all NaNs with 12PM
#Note that we can't use time to measure effectiveness of spray now, if that ever comes up


spray.fillna('12:00:00 PM', inplace=True)

In [72]:
#did not use this cell as it was adding the current date to the time
#spray.Time = pd.to_datetime(spray.Time)


In [73]:
spray.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14835 entries, 0 to 14834
Data columns (total 4 columns):
Date         14835 non-null datetime64[ns]
Time         14835 non-null object
Latitude     14835 non-null float64
Longitude    14835 non-null float64
dtypes: datetime64[ns](1), float64(2), object(1)
memory usage: 463.7+ KB


In [74]:
#10 sprays total
spray.Date.value_counts()

2013-08-15    2668
2013-08-29    2302
2013-07-17    2202
2011-09-07    2114
2013-07-25    1607
2013-08-22    1587
2013-08-08    1195
2013-09-05     924
2013-08-16     141
2011-08-29      95
Name: Date, dtype: int64

In [75]:
spray.set_index('Date', inplace=True)

In [76]:
spray_centers = spray.groupby('Date').mean()

In [77]:
#Because this is showing the mean lat/long, I think it's safe to call this the center spray point.  
#It may be useful to get the boundaries of the spray too though

spray_centers

,Latitude,Longitude
Date,,
2011-08-29,42.392331,-88.093370
2011-09-07,41.983344,-87.805927
2013-07-17,41.798570,-87.681137
2013-07-25,41.954535,-87.722879
2013-08-08,41.931466,-87.697035
2013-08-15,41.926007,-87.783171
2013-08-16,41.957809,-87.780646
2013-08-22,41.814052,-87.666029
2013-08-29,41.860128,-87.703237


In [78]:
#These could come in handy if we wanted to map the area covered by each spray later 
#(like the states were mapped in class, or on tableau)

min_latitude = []
max_latitude = []
min_longitude = []
max_longitude = []

for index in spray_centers.index:
    min_latitude.append(spray[(spray.index == index)].Latitude.min())
    max_latitude.append(spray[(spray.index == index)].Latitude.max())
    min_longitude.append(spray[(spray.index == index)].Longitude.min())
    max_longitude.append(spray[(spray.index == index)].Longitude.max())
    

In [79]:
spray_centers['min_latitude'] = min_latitude
spray_centers['max_latitude'] = max_latitude
spray_centers['min_longitude'] = min_longitude
spray_centers['max_longitude'] = max_longitude

In [80]:
spray_centers

,Latitude,Longitude,min_latitude,max_latitude,min_longitude,max_longitude
Date,,,,,,
2011-08-29,42.392331,-88.093370,42.389460,42.395983,-88.096468,-88.087988
2011-09-07,41.983344,-87.805927,41.968435,41.997028,-87.836560,-87.787710
2013-07-17,41.798570,-87.681137,41.714098,42.015098,-87.871023,-87.629355
2013-07-25,41.954535,-87.722879,41.939308,41.968097,-87.739840,-87.707987
2013-08-08,41.931466,-87.697035,41.917227,41.946703,-87.716847,-87.678060
2013-08-15,41.926007,-87.783171,41.887825,41.966532,-87.818408,-87.745625
2013-08-16,41.957809,-87.780646,41.911375,41.964712,-87.785792,-87.746675
2013-08-22,41.814052,-87.666029,41.713925,41.895475,-87.730655,-87.586727
2013-08-29,41.860128,-87.703237,41.758812,41.997808,-87.741315,-87.660885


In [81]:
#Some days, spraying was concentrated in one area, other days it spread in one or both directions

In [82]:
#This was cool, even though the value_counts are sorted, they lined up to the proper indexes.

spray_centers['Total_sprays'] = spray.index.value_counts()

In [83]:
spray_centers

,Latitude,Longitude,min_latitude,max_latitude,min_longitude,max_longitude,Total_sprays
Date,,,,,,,
2011-08-29,42.392331,-88.093370,42.389460,42.395983,-88.096468,-88.087988,95
2011-09-07,41.983344,-87.805927,41.968435,41.997028,-87.836560,-87.787710,2114
2013-07-17,41.798570,-87.681137,41.714098,42.015098,-87.871023,-87.629355,2202
2013-07-25,41.954535,-87.722879,41.939308,41.968097,-87.739840,-87.707987,1607
2013-08-08,41.931466,-87.697035,41.917227,41.946703,-87.716847,-87.678060,1195
2013-08-15,41.926007,-87.783171,41.887825,41.966532,-87.818408,-87.745625,2668
2013-08-16,41.957809,-87.780646,41.911375,41.964712,-87.785792,-87.746675,141
2013-08-22,41.814052,-87.666029,41.713925,41.895475,-87.730655,-87.586727,1587
2013-08-29,41.860128,-87.703237,41.758812,41.997808,-87.741315,-87.660885,2302


#### Train/Test

'train.csv, test.csv - the training and test set of the main dataset. The training set consists of data from 2007, 2009, 2011, and 2013, while in the test set you are requested to predict the test results for 2008, 2010, 2012, and 2014.' - kaggle

In [84]:
train.head(1)

,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy,NumMosquitos,WnvPresent
0,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9,1,0


In [85]:
test.head(1)

,Id,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy
0,1,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9


##### adding zipcodes.

In [86]:
## Looked through the addresses, and many do not have a zip.
## Going to loop through and see what kind of values come back

zipcodes = []
for add in train.Address:
    zipcodes.append(add[-10:-5])

In [87]:
#returned 51 zip codes, some are missing zips though

len(set(zipcodes))

51

In [88]:
#14.3 percent of zipcodes will be missing.  I think we can still dummy them out and 
#get value in the model, but I'll hold off on dummies for now.

zipcodes.count('o, IL')/len(zipcodes)

0.14353702646106986

In [89]:
zipcodes = ['Unk' if x == 'o, IL' else x for x in zipcodes]

In [90]:
train.insert(loc=2, column='zipcode', value=zipcodes)

In [91]:
train.head(1)

,Date,Address,zipcode,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy,NumMosquitos,WnvPresent
0,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",60634,CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9,1,0


In [92]:
#repeating for test

zipcodes2 = []
for add in test.Address:
    zipcodes2.append(add[-10:-5])

In [93]:
#returned 51 zip codes, some are missing zips though.
#hopefully they're the same 51 zips for when we dummy.  we'll cross that bridge later.

len(set(zipcodes2))

51

In [94]:
zipcodes2.count('o, IL')/len(zipcodes2)

0.08542216642446235

In [95]:
zipcodes2 = ['Unk' if x == 'o, IL' else x for x in zipcodes2]

In [96]:
test.insert(loc=2, column='zipcode', value=zipcodes2)

In [97]:
test.head(1)

,Id,Date,zipcode,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy
0,1,2008-06-11,60634,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9


In [98]:
train.drop(['Address', 'Street', 'AddressNumberAndStreet'], axis=1, inplace=True)

In [99]:
train.head(2)

,Date,zipcode,Species,Block,Trap,Latitude,Longitude,AddressAccuracy,NumMosquitos,WnvPresent
0,2007-05-29,60634,CULEX PIPIENS/RESTUANS,41,T002,41.95469,-87.800991,9,1,0
1,2007-05-29,60634,CULEX RESTUANS,41,T002,41.95469,-87.800991,9,1,0


In [100]:
test.drop(['Address', 'Street', 'AddressNumberAndStreet'], axis=1, inplace=True)

In [101]:
test.head(2)

,Id,Date,zipcode,Species,Block,Trap,Latitude,Longitude,AddressAccuracy
0,1,2008-06-11,60634,CULEX PIPIENS/RESTUANS,41,T002,41.95469,-87.800991,9
1,2,2008-06-11,60634,CULEX RESTUANS,41,T002,41.95469,-87.800991,9


In [102]:
train = pd.get_dummies(train, columns=['Species', 'zipcode', 'Trap']) 

In [103]:
test = pd.get_dummies(test, columns=['Species', 'zipcode', 'Trap']) 

In [104]:
train.shape

(10506, 201)

In [105]:
test.shape

(116293, 214)

In [106]:
(set(test.columns)) - (set(train.columns))

{'Id',
 'Species_UNSPECIFIED CULEX',
 'Trap_T002A',
 'Trap_T002B',
 'Trap_T065A',
 'Trap_T090A',
 'Trap_T090B',
 'Trap_T090C',
 'Trap_T128A',
 'Trap_T200A',
 'Trap_T200B',
 'Trap_T218A',
 'Trap_T218B',
 'Trap_T218C',
 'Trap_T234'}

In [107]:
train['Species_UNSPECIFIED CULEX'] = [0] * train.shape[0]
train['Trap_T002A'] = [0] * train.shape[0]
train['Trap_T002B'] = [0] * train.shape[0]
train['Trap_T065A'] = [0] * train.shape[0]
train['Trap_T090A'] = [0] * train.shape[0]
train['Trap_T090B'] = [0] * train.shape[0]
train['Trap_T090C'] = [0] * train.shape[0]
train['Trap_T128A'] = [0] * train.shape[0]
train['Trap_T200A'] = [0] * train.shape[0]
train['Trap_T200B'] = [0] * train.shape[0]
train['Trap_T218A'] = [0] * train.shape[0]
train['Trap_T218B'] = [0] * train.shape[0]
train['Trap_T218C'] = [0] * train.shape[0]
train['Trap_T234'] = [0] * train.shape[0]

In [108]:
train.shape

(10506, 215)

In [109]:
test.shape

(116293, 214)

In [112]:
(set(test.columns)) - (set(train.columns))

{'Id'}

In [113]:
(set(train.columns)) - (set(test.columns))

{'NumMosquitos', 'WnvPresent'}

In [118]:
train.to_csv('west_nile_train.csv')

In [ ]:
test.to_csv('west_nile_test.csv')

^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
train test cleaned, dummied
---------


___________

---------

anything under here is unstructured/undone/meaningless

-----------

In [37]:
len(train.Trap.value_counts())
#len(train.Trap.unique()) samesame

136

In [38]:
len(test.Trap.value_counts())

149

In [39]:
train.sort_values(by='NumMosquitos', ascending = False).head()

,Date,Address,zipcode,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy,NumMosquitos,WnvPresent
1233,2007-08-01,"South Doty Avenue, Chicago, IL, USA",Unk,CULEX PIPIENS,12,S DOTY AVE,T115,"1200 S DOTY AVE, Chicago, IL",41.673408,-87.599862,5,50,0
1406,2007-08-02,"South Stony Island Avenue, Chicago, IL, USA",Unk,CULEX PIPIENS,10,S STONY ISLAND AVE,T138,"1000 S STONY ISLAND AVE, Chicago, IL",41.726465,-87.585413,5,50,0
4284,2009-06-22,"ORD Terminal 5, O'Hare International Airport, ...",60666,CULEX PIPIENS/RESTUANS,10,W OHARE AIRPORT,T900,"1000 W OHARE AIRPORT, Chicago, IL",41.974689,-87.890615,9,50,0
4285,2009-06-22,"ORD Terminal 5, O'Hare International Airport, ...",60666,CULEX PIPIENS/RESTUANS,10,W OHARE AIRPORT,T900,"1000 W OHARE AIRPORT, Chicago, IL",41.974689,-87.890615,9,50,0
4287,2009-06-22,"ORD Terminal 5, O'Hare International Airport, ...",60666,CULEX PIPIENS/RESTUANS,10,W OHARE AIRPORT,T900,"1000 W OHARE AIRPORT, Chicago, IL",41.974689,-87.890615,9,50,0


In [40]:
#Species breakdown when WNV present
#May not be strong enough because of low count of other species.

train[(train.WnvPresent == 1)].Species.value_counts()

CULEX PIPIENS/RESTUANS    262
CULEX PIPIENS             240
CULEX RESTUANS             49
Name: Species, dtype: int64

In [41]:
#total Species breakdown wnv

train.Species.value_counts()

CULEX PIPIENS/RESTUANS    4752
CULEX RESTUANS            2740
CULEX PIPIENS             2699
CULEX TERRITANS            222
CULEX SALINARIUS            86
CULEX TARSALIS               6
CULEX ERRATICUS              1
Name: Species, dtype: int64

In [42]:
#test.Species is balanced

test.Species.value_counts()

CULEX PIPIENS/RESTUANS    15359
CULEX RESTUANS            14670
CULEX PIPIENS             14521
CULEX SALINARIUS          14355
CULEX TERRITANS           14351
CULEX TARSALIS            14347
CULEX ERRATICUS           14345
UNSPECIFIED CULEX         14345
Name: Species, dtype: int64

In [43]:
len(train.Block.value_counts())

64

In [44]:
len(test.Block.value_counts())

65

In [45]:
len(weather.Date.unique())

1472